## Aggregating ACS, FCC, Chicago Community Area Data

###### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

###### Importing Chicago FCC data

First, we use the FCC IL Dec 2020 csv file (which can be found [here](https://us-fcc.app.box.com/v/IL-Dec2020-v1)). Since it is a large file and we only want data on Chicago, we filter for Cook County data only and then export it as a new csv file which is saved in the "data" folder. **DO NOT RUN THE FOLLOWING CHUNKS OF CODE!!!**

In [ ]:
# FCC IL

fcc_df = pd.read_csv("data/IL-Fixed-Dec2020-v1.csv",
                     index_col=0,parse_dates=[0])

In [ ]:
#changing BlockCode column to string type

fcc_df['BlockCode']=fcc_df['BlockCode'].astype(str)

In [ ]:
# Extracting state, county, tract, block numbers from BlockCode column
# IL state=17
# Cook County=031

fcc_df['state'] = fcc_df['BlockCode'].str[:2]
fcc_df['county'] = fcc_df['BlockCode'].str[2:5]
fcc_df['tract'] = fcc_df['BlockCode'].str[5:11]
fcc_df['block'] = fcc_df['BlockCode'].str[-4:]

In [ ]:
# Filtering for Cook County only 
# 763788 rows

chi_fcc = fcc_df[(fcc_df.county == "031")]

In [ ]:
# dropping columns we don't need to make the file smaller

chi_fcc=chi_fcc[['ProviderName', 'Consumer', 
                 'MaxAdDown','MaxAdUp','tract']]
chi_fcc['tract']=chi_fcc['tract'].astype(float)

In [ ]:
# export final dataframe to csv file

chi_fcc.to_csv(r'data/chi_fcc.csv', index = False)

In [ ]:
# final dataframe looks like this
chi_fcc.head(5)

Run this code to load the Chicago FCC data.  

In [2]:
# Chicago FCC data

chi_fcc = pd.read_csv("data/chi_fcc.csv",index_col=0,parse_dates=[0])
chi_fcc.head(5)

,Consumer,MaxAdDown,MaxAdUp,tract
ProviderName,,,,
"TOWERSTREAM, INC.",0,0.0,0.0,20702.0
"TOWERSTREAM, INC.",0,0.0,0.0,80300.0
"TOWERSTREAM, INC.",0,0.0,0.0,81000.0
"TOWERSTREAM, INC.",0,0.0,0.0,81300.0
"TOWERSTREAM, INC.",0,0.0,0.0,81500.0


###### Importing Chicago ACS aggregate and profile data

In [3]:
# ACS aggregate

acs_agg = pd.read_csv("data/acs5_aggregate.csv",index_col=0,
                      parse_dates=[0]).drop(['state', 'county'], axis=1)

In [4]:
# ACS profile

acs_pro = pd.read_csv("data/acs5_profile.csv",
                      index_col=0,
                      parse_dates=[0]).drop(
                        ['state', 'county','DP05_0001E'], axis=1)

In [63]:
# merging both ACS datasets
# renaming variables
# 1319 rows x 19 columns

acs_df = acs_agg.merge(acs_pro, on='tract').rename(columns={
                                'B01003_001E': 'total_pop',
                                'B28002_004E': 'has_internet',
                               'B28002_013E': 'no_internet',
                               'B28003_002E': 'has_computer',
                               'B28003_004E': 'has_computer_broad',
                               'B28003_005E': "has_computer_no_internet",
                               'B28003_006E': 'no_computer',
                               'DP02_0001E': 'total_households',
                               'DP02_0152E': 'total_house_computer',
                               'DP02_0152PE': 'percent_house_computer',
                               'DP02_0153E': 'has_broadband',
                               'DP02_0153PE': 'percent_house_broad',
                               'DP03_0062E': 'median_income',
                               'DP03_0119PE': 'percent_poverty',
                               'DP05_0071E': 'total_hispanic',
                               'DP05_0071PE': 'percent_hispanic',
                                'DP05_0078E': 'total_black',
                                'DP05_0078PE': 'percent_black'})

acs_df.head(5)

,total_pop,has_internet,no_internet,has_computer,has_computer_broad,has_computer_no_internet,no_computer,tract,total_households,total_house_computer,percent_house_computer,has_broadband,percent_house_broad,median_income,percent_poverty,total_hispanic,percent_hispanic,total_black,percent_black
0,1825,392,149,426,392,34,149,630200,575,426,74.1,392,68.2,37422,25.7,1622,88.9,0,0.0
1,5908,1242,231,1411,1242,169,133,580700,1544,1411,91.4,1242,80.4,47000,17.4,4742,80.3,161,2.7
2,3419,917,140,1068,917,140,104,590600,1172,1068,91.1,917,78.2,46033,7.9,2119,62.0,9,0.3
3,2835,917,138,1003,917,86,81,600700,1084,1003,92.5,917,84.6,45294,17.0,850,30.0,82,2.9
4,1639,322,245,356,322,34,218,611900,574,356,62.0,322,56.1,24507,55.0,438,26.7,1175,71.7


###### Importing Chicago Community Area data

In [6]:
# Chi comm area
#rename columns

comm_area = pd.read_csv("data/chi_tracts.csv",
                        index_col=0,
                        parse_dates=[0]).rename(columns={
                        "Community Area Name": "name",          
                        "TRACT": "tract"})

In [7]:
# selecting columns we need 

comm_area=comm_area[['name', 'tract']]
comm_area.head(5)

,name,tract
STUSAB,,
IL,Rogers Park,10100
IL,Rogers Park,10201
IL,Rogers Park,10202
IL,Rogers Park,10300
IL,Rogers Park,10400


Joining ACS data and Community Areas

In [64]:
# 803 rows x 20 columns

acs_comm = comm_area.merge(acs_df, on='tract')
acs_comm.head(5)

,name,tract,total_pop,has_internet,no_internet,has_computer,has_computer_broad,has_computer_no_internet,no_computer,total_households,total_house_computer,percent_house_computer,has_broadband,percent_house_broad,median_income,percent_poverty,total_hispanic,percent_hispanic,total_black,percent_black
0,Rogers Park,10100,4599,1789,460,2044,1762,270,319,2363,2044,86.5,1789,75.7,32474,26.5,523,11.4,2045,44.5
1,Rogers Park,10201,7455,2091,434,2469,2091,378,278,2747,2469,89.9,2091,76.1,45639,28.6,1671,22.4,2481,33.3
2,Rogers Park,10202,2896,919,176,1022,912,110,114,1136,1022,90.0,919,80.9,41486,16.4,753,26.0,974,33.6
3,Rogers Park,10300,6485,2620,418,2883,2604,279,208,3091,2883,93.3,2620,84.8,41250,6.6,1099,16.9,1995,30.8
4,Rogers Park,10400,5213,1611,333,1774,1611,163,214,1988,1774,89.2,1611,81.0,39700,13.9,392,7.5,1104,21.2


Joining FCC data and Community Areas

In [9]:
# 372924 rows × 5 columns

fcc_comm = comm_area.merge(chi_fcc, on='tract')
fcc_comm.head(5)

,name,tract,Consumer,MaxAdDown,MaxAdUp
0,Rogers Park,10100,0,0.0,0.0
1,Rogers Park,10100,0,0.0,0.0
2,Rogers Park,10100,0,0.0,0.0
3,Rogers Park,10100,0,0.0,0.0
4,Rogers Park,10100,0,0.0,0.0


###### Joining all datasets by tract

In [ ]:
# merging all three dataframes by the column "tract"
# 372924 rows x 23 columns

full_df = comm_area.merge(acs_df,
                        on='tract').merge(chi_fcc,
                                          on='tract')


In [ ]:
full_df.head(5)

In [ ]:
full_df[['name', 'tract',
        'b_total_pop']].groupby(['tract', 'name']).mean()

###### Computer, Internet, Broadband Access

This section will look into the community areas at a household-level to see who has and who does not have basic access to an internet and/or computer. We will also look into the households who have a broadband internet subscription. 

In [76]:
# who has a computer? who has internet access? 
# selecting columns we need

internet_df = acs_comm[['name', 
                        'total_pop',
                        'total_households',
                        'no_internet', 
                        'has_internet', 
                        'has_computer', 
                        'no_computer',
                       'has_broadband']].groupby(by="name").sum()

In [77]:
internet_df.head(5)

,total_pop,total_households,no_internet,has_internet,has_computer,no_computer,has_broadband
name,,,,,,,
Albany Park,49806,16909,2674,13477,15104,1805,13477
"Archer Heights,",13726,3919,772,2886,3207,712,2886
Armour Square,13538,5396,1488,3674,4064,1332,3674
Ashburn,43356,13124,1840,10442,11847,1277,10442
Auburn Gresham,45909,17161,5282,10291,13724,3437,10291


In [79]:
# calculating who does not have a broadband internet subscription

internet_df['no_broadband']=internet_df['total_households']-internet_df['has_broadband']

In [80]:
# calculating percentages of households for each variable

# percentage of households with/out internet access

internet_df['no_internet']=internet_df['no_internet']/internet_df['total_households']*100
internet_df['has_internet']=internet_df['has_internet']/internet_df['total_households']*100

# percentage of households with/out no computer

internet_df['no_computer']=internet_df['no_computer']/internet_df['total_households']*100
internet_df['has_computer']=internet_df['has_computer']/internet_df['total_households']*100

# percentage of households with/out broadband 

internet_df['no_broadband']=internet_df['no_broadband']/internet_df['total_households']*100
internet_df['has_broadband']=internet_df['has_broadband']/internet_df['total_households']*100

In [89]:
# resulting dataframe
# yellow indicates the maximum values in each column
# light blue indicates the minimum values in each column

internet_df.sort_values(["has_internet"],
                        ascending=True).style.highlight_max(color='yellow').highlight_min(color = 'lightblue')

,total_pop,total_households,no_internet,has_internet,has_computer,no_computer,has_broadband,no_broadband
name,,,,,,,,
Burnside,2006,888,42.229730,52.252252,62.725225,37.274775,52.252252,47.747748
Englewood,22921,8983,34.598686,56.306356,74.897028,25.102972,56.306356,43.693644
West Englewood,26346,9483,38.089212,57.239270,70.336391,29.663609,57.239270,42.760730
Fuller Park,2330,1128,34.663121,59.042553,69.414894,30.585106,59.042553,40.957447
North Lawndale,32073,11075,29.986456,59.620767,76.000000,24.000000,59.620767,40.379233
Auburn Gresham,45909,17161,30.779092,59.967368,79.972030,20.027970,59.967368,40.032632
East Garfield Park,18940,6794,31.630851,60.980277,77.038563,22.961437,60.980277,39.019723
West Garfield Park,16415,5353,32.617224,61.124603,75.602466,24.397534,61.124603,38.875397
Greater Grand Crossing,30149,12134,31.646613,63.095434,79.866491,20.133509,63.095434,36.904566


Based on 2015-2019 ACS data, the percentages of households with computers, internet access, and internet broadband subscription are seen above. The neighborhoods of Burnside, Englewood, West Englewood, Fuller Park have the lowest percentages of both broadband subscription and internet access. The neighborhods of Near South Side, Lake View, Lincoln Park, and North Center have the highest. 

##### Average and Median Download and Upload Speeds by Chicago Community Area

In [90]:
# selecting columns we need

fcc_comm

,name,tract,Consumer,MaxAdDown,MaxAdUp
0,Rogers Park,10100,0,0.0,0.0
1,Rogers Park,10100,0,0.0,0.0
2,Rogers Park,10100,0,0.0,0.0
3,Rogers Park,10100,0,0.0,0.0
4,Rogers Park,10100,0,0.0,0.0
...,...,...,...,...,...
372919,Edgewater,30900,1,2.0,1.3
372920,Edgewater,30900,1,2.0,1.3
372921,Edgewater,30900,1,2.0,1.3
372922,Edgewater,30900,1,2.0,1.3


In [91]:
#calculating mean speeds

fcc_comm.groupby(by="name").mean().sort_values(["MaxAdUp"], 
                                                      ascending=False)

,tract,Consumer,MaxAdDown,MaxAdUp
name,,,,
Dunning,170645.873895,0.860416,246.429453,115.015010
West Town,296281.829701,0.836374,229.364653,111.663583
Avondale,327857.299776,0.829754,228.131013,105.684701
Irving Park,160685.726342,0.840700,230.232691,105.280061
Garfield Ridge,585019.264448,0.843695,232.221788,102.162929
...,...,...,...,...
Gage Park,665833.101909,0.818745,147.880358,7.766088
West Englewood,680402.854215,0.834309,147.488897,7.525122
West Garfield Park,260606.682659,0.837560,134.083025,7.127243


In [92]:
# calculating median speeds

fcc_comm.groupby(by="name").median().sort_values(["MaxAdUp"], 
                                                      ascending=False)

,tract,Consumer,MaxAdDown,MaxAdUp
name,,,,
Albany Park,140500.0,1.0,25.0,3.0
Pullman,500100.0,1.0,25.0,3.0
Ohare,770602.0,1.0,25.0,3.0
Oakland,836400.0,1.0,25.0,3.0
Norwood Park,100500.0,1.0,25.0,3.0
...,...,...,...,...
Englewood,681300.0,1.0,25.0,3.0
Edison Park,90200.0,1.0,25.0,3.0
Near West Side,833100.0,1.0,18.0,1.5


This was interesting because the means and medians of average download / upload speeds were very different. The average download speeds were in the range of 114 to 293 mbps. The average upload speeds were from 7 to 115 mbps. The median upload speeds were 25 mbps and download speeds were 3 mbps for all community areas.

###### Median Household Income

In [ ]:
# selecting columns we need

median = full_df[['name', 'median_income']]
median.groupby(by = "name").median().sort_values(["median_income"], 
                                                      ascending = False)

The median household income for the neighborhoods is in the range of $15,293 to $145,855. The lowest is in Riverdale and the highest in Lincoln Park.

###### Household Poverty Rate

In [ ]:
# selecting columns we need

poverty = full_df[['name', 'percent_poverty']]
poverty.groupby(by = "name").median().sort_values(["percent_poverty"], 
                                                      ascending = False)

Riverdale has the highest percent of poverty with 53.3% and Lake View is 1.4%.